In [1]:
import numpy as np
from dipy.io.image import load_nifti, save_nifti
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.segment.mask import median_otsu
from dipy.reconst.dti import fractional_anisotropy, color_fa

import dipy.reconst.dti as dti
import nibabel as nib

In [15]:
data, affine = load_nifti('/Users/jdrussell3/scratch/dtimodel/sub-001_ses-01_ppd.nii')
bvals, bvecs = read_bvals_bvecs('/Users/jdrussell3/scratch/dtimodel/sub-001_ses-01_ppd.bval',
                                '/Users/jdrussell3/scratch/dtimodel/sub-001_ses-01_ppd.bvec')
gtab = gradient_table(bvals, bvecs)
values = np.array(bvals)
ii = np.where(values == bvals.min())[0]
b0_average = np.mean(data[:,:,:,ii], axis=3)



In [17]:
maskdata = nib.load('/Users/jdrussell3/scratch/dtimodel/sub-001_ses-01_mask_ppd.nii').get_fdata()

dti_model = dti.TensorModel(gtab)

dti_fit = tenmodel.fit(data, maskdata)
evals = dti_fit.evals.astype(np.float32)
evecs = dti_fit.evecs.astype(np.float32)

FA = fractional_anisotropy(dti_fit.evals)
FA[np.isnan(FA)] = 0

estimate_data = dti_fit.predict(gtab, S0=b0_average)




In [18]:
residuals = np.absolute(data - estimate_data)

In [12]:

save_nifti('tensor_fa.nii.gz', FA.astype(np.float32), affine)

# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2041910/
# diffusion tensor may be visualized using an ellipsoid with the 
# eigenvectors defining the directions of the principle axes and 
# the ellipsoidal radii (apparent diffusivity in each direction)
# defined by the eigenvalues

# Diffusion is isotropic when L1 ~ L2 ~ L3, and anisotropic when L1 > L2 > L3

# Eigenvector V1
dti_V1 = evecs[:,:,:,:,0]
save_nifti('tensor_V1.nii.gz', dti_V1.astype(np.float32), affine)

# Eigenvector V2
dti_V2 = evecs[:,:,:,:,1]
save_nifti('tensor_V2.nii.gz', dti_V2.astype(np.float32), affine)

# Eigenvector V3
dti_V3 = evecs[:,:,:,:,2]
save_nifti('tensor_V3.nii.gz', dti_V3.astype(np.float32), affine)

# Eigenvalue L1
dti_L1 = evals[:,:,:,0]
save_nifti('tensor_L1.nii.gz', dti_L1.astype(np.float32), affine)

# Eigenvalue L2
dti_L2 = evals[:,:,:,0]
save_nifti('tensor_L2.nii.gz', dti_L2.astype(np.float32), affine)

# Eigenvalue L3
dti_L3 = evals[:,:,:,0]
save_nifti('tensor_L3.nii.gz', dti_L3.astype(np.float32), affine)

res_img